In [81]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from joblib import Parallel, delayed
from multiprocessing import cpu_count

In [95]:
# from shutil import copyfile
import os

In [6]:
path_ncbi = "/home/ubuntu/Data/NCBI/Bacteria_2015"

In [94]:
path_DB_splits = "/home/ubuntu/SSD/NCBI_bins/10_clusters_v1"

In [3]:
path_splits = '/home/ubuntu/Data/kmer_freq/4mer/V4/clustering/genomes_split_10_LenScale.pd'
genomes_split = pd.read_pickle(path_splits)

In [48]:
window_size = 10000

In [84]:
n_cores = cpu_count()

combine the splits if same cluster

In [5]:
genomes_split

,bacteria,fna,start,cluster
0,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,0,7
1,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,10000,7
2,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,20000,2
3,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,30000,7
4,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,40000,7
5,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,50000,7
6,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,60000,7
7,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,70000,7
8,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,80000,7
9,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,90000,7


In [24]:
split_file = []
split_fna = []
split_cluster = []
split_start = []
split_end = []
a = 0

for i, df in tqdm(genomes_split.groupby([(genomes_split.cluster != genomes_split.cluster.shift()).cumsum()])):
    split_file.append(df.bacteria.iloc[0])
    split_fna.append(df.fna.iloc[0])
    split_cluster.append(df.cluster.iloc[0])
    split_start.append(df.start.iloc[0])
    split_end.append(df.start.iloc[-1] + window_size)

    a += 1
    if a > 1000000: break

In [56]:
file_split = pd.DataFrame({
    "file": split_file, 
    "fna": split_fna, 
    "cluster": split_cluster, 
    "start": split_start, 
    "end": split_end, 
})

In [75]:
file_split

,file,fna,cluster,start,end
0,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,7,0,20000
1,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,2,20000,30000
2,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,7,30000,110000
3,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,0,110000,130000
4,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,7,130000,420000
5,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,0,420000,430000
6,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,7,430000,460000
7,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,0,460000,470000
8,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,7,470000,870000
9,Propionibacterium_acnes_KPA171202_uid58101,NC_006085,2,870000,880000


In [72]:
path_splits_combined = '/home/ubuntu/Data/kmer_freq/4mer/V4/clustering/genomes_splits_combined.pd'

In [73]:
file_split.to_pickle(path_splits_combined)

Need to copy parts of files into various folders for Kraken

In [91]:
def read_fna(file_path):
    with open(file_path) as f:
        rec = f.readlines()
        return "".join(rec[1:]).replace("\n", "")

In [93]:
def copy_split(file, start, end, cluster, src=path_ncbi, dst=path_splits):
    path_dir = os.path.join(dst, str(cluster), file) + "/"
    os.makedirs(path_dir, exist_ok=True)
    

In [74]:
file_split = pd.read_pickle(path_splits_combined)

In [79]:
a = 0
for i, b in tqdm(file_split.groupby([(file_split.fna != file_split.fna.shift()).cumsum()])):
    print(i)
    print(b)
    for
    
    a += 1
    if a > 1: break

1
                                          file        fna  cluster    start  \
0   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        7        0   
1   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        2    20000   
2   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        7    30000   
3   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        0   110000   
4   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        7   130000   
5   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        0   420000   
6   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        7   430000   
7   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        0   460000   
8   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        7   470000   
9   Propionibacterium_acnes_KPA171202_uid58101  NC_006085        2   870000   
10  Propionibacterium_acnes_KPA171202_uid58101  NC_006085        7   880000   
11  Propionibacterium_acnes_KPA171202_uid58101  NC

Need to loop through each folder/bacteria, through each .fna file, copy only the fragment that is part of the cluster.
Need to check what does Kraken need to build the taxonomy ID

In [ ]:
Parallel(n_jobs=n_cores)(delayed(try_classifier)(i+1, name, clf) \
                   for i, (name, cls) in tqdm(enumerate(zip(names, classifiers)), total=len(names)))


In [90]:
a = 0
for i in file_split.values:
    print(i)
    
    a += 1
    if a > 10: break

['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 7 0 20000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 2 20000 30000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 7 30000 110000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 0 110000 130000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 7 130000 420000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 0 420000 430000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 7 430000 460000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 0 460000 470000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 7 470000 870000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 2 870000 880000]
['Propionibacterium_acnes_KPA171202_uid58101' 'NC_006085' 7 880000 920000]
